# آزمایش-۱-معرفی فی-۳.۵ دستور


### تبدیل مدل Phi-3-mini-4k-instruct به INT4 با استفاده از optimum-cli


In [ ]:
from pathlib import Path

llm_model_id = "microsoft/Phi-3.5-mini-instruct"
llm_model_path = "../model/phi-3.5-mini-instruct-ov"

if not Path(llm_model_path).exists():
    !optimum-cli export openvino --model {llm_model_id} --task text-generation-with-past --weight-format int4 --group-size 128 --ratio 0.6  --sym  --trust-remote-code {llm_model_path}

In [ ]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

In [ ]:
ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

In [ ]:
ov_model = OVModelForCausalLM.from_pretrained(
    llm_model_path,
    device='GPU',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(llm_model_path, trust_remote_code=True),
    trust_remote_code=True,
)

In [ ]:
tok = AutoTokenizer.from_pretrained(llm_model_path, trust_remote_code=True)

In [ ]:
tokenizer_kwargs =  {"add_special_tokens": False}

In [ ]:
prompt = "<|user|>\n你了解 .NET 吗?\n<|end|><|assistant|>\n"
input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

In [ ]:
answer = ov_model.generate(**input_tokens, max_new_tokens=1024)

In [ ]:
tok.batch_decode(answer, skip_special_tokens=True)[0]


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
